# FGSD Graph Embedding Evaluation on REDDIT-MULTI-12K Dataset

This notebook evaluates the FGSD (Family of Graph Spectral Distances) method on the REDDIT-MULTI-12K dataset using various classifiers.

**Requirements:**
- Google Colab (GPU runtime recommended)
- Internet connection for dataset download

**Runtime Setup:**
1. Go to Runtime → Change runtime type
2. Select GPU as Hardware accelerator
3. Click Save

## 1. Environment Setup and Dependencies Installation

In [ ]:
# Install required packages with compatible versions for Google Colab
!pip install -q "numpy>=1.26.0,<2.2.0"
!pip install -q "scipy>=1.11.0"
!pip install -q "networkx>=3.0"
!pip install -q "scikit-learn>=1.3.0"
!pip install -q pandas
!pip install -q matplotlib
!pip install -q seaborn

print("✓ All dependencies installed successfully!")

In [ ]:
# Import libraries
import numpy as np
import time
import tracemalloc
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize, StandardScaler
import sklearn
import networkx as nx
import warnings
import os
import urllib.request
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"NetworkX version: {nx.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

## 2. FGSD Implementation

In [ ]:
from typing import List

class Estimator:
    """Base estimator class."""
    def _set_seed(self):
        np.random.seed(self.seed)
    
    def _check_graphs(self, graphs):
        return graphs

class FGSD(Estimator):
    r"""An implementation of FGSD from NeurIPS '17.
    
    The procedure calculates the Moore-Penrose spectrum of the normalized Laplacian.
    Using this spectrum the histogram of the spectral features is used as a whole graph representation.

    Args:
        hist_bins (int): Number of histogram bins. Default is 200.
        hist_range (int): Histogram range considered. Default is 20.
        seed (int): Random seed value. Default is 42.
        regularization (float): Regularization factor for numerical stability. Default is 1e-10.
    """

    def __init__(self, hist_bins: int = 200, hist_range: int = 20, seed: int = 42, regularization: float = 1e-10):
        self.hist_bins = hist_bins
        self.hist_range = (0, hist_range)
        self.seed = seed
        self.regularization = regularization
        self.failed_indices = []

    def _calculate_fgsd(self, graph, graph_idx=None):
        """Calculate the features of a graph with error handling."""
        try:
            L = nx.normalized_laplacian_matrix(graph).todense()
            
            # Add regularization for numerical stability
            L_reg = L + self.regularization * np.eye(L.shape[0])
            
            # Try standard pinv first
            try:
                fL = np.linalg.pinv(L_reg, rcond=1e-10)
            except np.linalg.LinAlgError:
                # Fallback: use eigenvalue decomposition with regularization
                if graph_idx is not None:
                    print(f"  ⚠️  SVD failed for graph {graph_idx}, using eigenvalue fallback...")
                eigenvalues, eigenvectors = np.linalg.eigh(L_reg)
                # Filter out near-zero eigenvalues
                threshold = 1e-10
                eigenvalues_inv = np.where(np.abs(eigenvalues) > threshold, 1.0 / eigenvalues, 0)
                fL = eigenvectors @ np.diag(eigenvalues_inv) @ eigenvectors.T
            
            ones = np.ones(L.shape[0])
            S = np.outer(np.diag(fL), ones) + np.outer(ones, np.diag(fL)) - 2 * fL
            
            # Ensure S values are non-negative (can have small negative values due to numerical errors)
            S = np.maximum(S, 0)
            
            hist, _ = np.histogram(S.flatten(), bins=self.hist_bins, range=self.hist_range)
            return hist
            
        except Exception as e:
            if graph_idx is not None:
                print(f"  ❌ Error processing graph {graph_idx}: {str(e)}")
                self.failed_indices.append(graph_idx)
            # Return zero histogram as fallback
            return np.zeros(self.hist_bins)

    def fit(self, graphs: List[nx.classes.graph.Graph]):
        """Fit FGSD model."""
        self._set_seed()
        graphs = self._check_graphs(graphs)
        self.failed_indices = []
        
        print(f"Processing {len(graphs)} graphs...")
        self._embedding = []
        for idx, graph in enumerate(graphs):
            if (idx + 1) % 500 == 0:
                print(f"  Processed {idx + 1}/{len(graphs)} graphs...")
            self._embedding.append(self._calculate_fgsd(graph, idx))
        
        if self.failed_indices:
            print(f"⚠️  Warning: {len(self.failed_indices)} graphs failed processing")

    def get_embedding(self) -> np.array:
        """Get the embedding of graphs."""
        return np.array(self._embedding)

    def infer(self, graphs: List[nx.classes.graph.Graph]) -> np.array:
        """Infer the embedding for a list of graphs."""
        self._set_seed()
        graphs = self._check_graphs(graphs)
        
        print(f"Inferring {len(graphs)} graphs...")
        embedding = []
        for idx, graph in enumerate(graphs):
            if (idx + 1) % 500 == 0:
                print(f"  Processed {idx + 1}/{len(graphs)} graphs...")
            embedding.append(self._calculate_fgsd(graph, idx))
        
        return np.array(embedding)

print("✓ FGSD class defined successfully!")

## 3. Dataset Download and Loading

In [ ]:
def download_and_load_reddit():
    """Download and load REDDIT-MULTI-12K dataset from TU Dortmund."""
    data_dir = '/content/REDDIT-MULTI-12K'
    os.makedirs(data_dir, exist_ok=True)
    
    base_url = 'https://www.chrsmrrs.com/graphkerneldatasets/REDDIT-MULTI-12K.zip'
    zip_path = os.path.join(data_dir, 'REDDIT-MULTI-12K.zip')
    
    # Download if not exists
    if not os.path.exists(os.path.join(data_dir, 'REDDIT-MULTI-12K')):
        print("📥 Downloading REDDIT-MULTI-12K dataset (this may take a while)...")
        urllib.request.urlretrieve(base_url, zip_path)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print("✓ Download complete.")
    else:
        print("✓ Dataset already downloaded.")
    
    # Parse dataset files
    dataset_path = os.path.join(data_dir, 'REDDIT-MULTI-12K')
    
    print("📂 Loading dataset files...")
    graph_indicator = np.loadtxt(os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_indicator.txt'), dtype=int)
    edges = np.loadtxt(os.path.join(dataset_path, 'REDDIT-MULTI-12K_A.txt'), dtype=int, delimiter=',')
    graph_labels = np.loadtxt(os.path.join(dataset_path, 'REDDIT-MULTI-12K_graph_labels.txt'), dtype=int)
    
    print("🔨 Building NetworkX graphs...")
    num_graphs = len(graph_labels)
    graphs = [nx.Graph() for _ in range(num_graphs)]
    
    # Add nodes
    for node_id, graph_id in enumerate(graph_indicator, start=1):
        graphs[graph_id - 1].add_node(node_id)
    
    # Add edges
    print(f"Adding {len(edges)} edges...")
    for i, edge in enumerate(edges):
        if i % 100000 == 0:
            print(f"  Processed {i}/{len(edges)} edges...")
        node1, node2 = edge
        graph_id = graph_indicator[node1 - 1]
        graphs[graph_id - 1].add_edge(node1, node2)
    
    # Relabel nodes to be contiguous starting from 0
    print("🏷️  Relabeling nodes...")
    graphs = [nx.convert_node_labels_to_integers(g) for g in graphs]
    
    # Convert labels to 0-indexed
    labels = graph_labels - 1
    
    print(f"✓ Dataset loaded: {len(graphs)} graphs")
    return graphs, labels

print("✓ Dataset loading function defined!")

## 4. Dataset Analysis Functions

In [ ]:
def analyze_dataset(graphs, labels):
    """Analyze the dataset to understand its properties."""
    print("\n" + "="*70)
    print("📊 DATASET ANALYSIS")
    print("="*70)
    print(f"Number of graphs: {len(graphs)}")
    print(f"Number of classes: {len(np.unique(labels))}")
    
    unique, counts = np.unique(labels, return_counts=True)
    print(f"\nClass distribution:")
    for label, count in zip(unique, counts):
        print(f"  Class {label}: {count} graphs ({100*count/len(labels):.2f}%)")
    
    # Sample graphs for statistics
    sample_size = min(1000, len(graphs))
    sample_indices = np.random.choice(len(graphs), sample_size, replace=False)
    sample_graphs = [graphs[i] for i in sample_indices]
    
    # Graph statistics
    num_nodes = [g.number_of_nodes() for g in sample_graphs]
    num_edges = [g.number_of_edges() for g in sample_graphs]
    densities = [nx.density(g) if g.number_of_nodes() > 1 else 0 for g in sample_graphs]
    
    print(f"\n📈 Graph statistics (based on {sample_size} samples):")
    print(f"  Nodes - Min: {min(num_nodes)}, Max: {max(num_nodes)}, Mean: {np.mean(num_nodes):.2f}, Std: {np.std(num_nodes):.2f}")
    print(f"  Edges - Min: {min(num_edges)}, Max: {max(num_edges)}, Mean: {np.mean(num_edges):.2f}, Std: {np.std(num_edges):.2f}")
    print(f"  Density - Min: {min(densities):.4f}, Max: {max(densities):.4f}, Mean: {np.mean(densities):.4f}")
    
    # Check connectivity
    connected = [nx.is_connected(g) for g in sample_graphs]
    print(f"  Connected graphs: {sum(connected)}/{len(sample_graphs)} ({100*sum(connected)/len(sample_graphs):.2f}%)")
    print("="*70 + "\n")
    
    # Visualize class distribution
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.bar(unique, counts)
    plt.xlabel('Class Label')
    plt.ylabel('Number of Graphs')
    plt.title('Class Distribution')
    plt.grid(axis='y', alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.hist(num_nodes, bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Number of Nodes')
    plt.ylabel('Frequency')
    plt.title('Distribution of Graph Sizes (Sample)')
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("✓ Analysis function defined!")

## 5. Classifier Evaluation Functions

In [ ]:
def evaluate_classifier(X_train, X_test, y_train, y_test, classifier_name, clf, use_scaling=True):
    """Evaluate a single classifier with optional feature scaling."""
    # Feature scaling
    if use_scaling:
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    else:
        X_train_scaled = X_train
        X_test_scaled = X_test
    
    # Training
    start_time = time.time()
    clf.fit(X_train_scaled, y_train)
    train_time = time.time() - start_time
    
    # Prediction
    start_time = time.time()
    y_pred = clf.predict(X_test_scaled)
    inference_time = time.time() - start_time
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # AUC
    try:
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        if hasattr(clf, 'predict_proba'):
            y_score = clf.predict_proba(X_test_scaled)
        elif hasattr(clf, 'decision_function'):
            y_score = clf.decision_function(X_test_scaled)
            if len(y_score.shape) == 1:
                y_score = y_score.reshape(-1, 1)
        else:
            y_score = None
        
        if y_score is not None and y_test_bin.shape[1] > 1:
            auc = roc_auc_score(y_test_bin, y_score, average='weighted', multi_class='ovr')
        else:
            auc = None
    except Exception as e:
        auc = None
    
    return {
        'classifier': classifier_name,
        'accuracy': accuracy,
        'f1_score': f1,
        'auc': auc,
        'train_time': train_time,
        'inference_time': inference_time,
        'y_pred': y_pred
    }

print("✓ Evaluation function defined!")

## 6. Load and Analyze Dataset

In [ ]:
# Load dataset
graphs, labels = download_and_load_reddit()

# Analyze dataset
analyze_dataset(graphs, labels)

## 7. Configuration and Data Preparation

In [ ]:
# Configuration
EMBEDDING_DIMENSIONS = [100, 200, 400]
TEST_SIZE = 0.2
RANDOM_STATE = 42
MAX_TRAIN_SIZE = 8000  # Use subset for faster experiments

print("⚙️  Configuration:")
print(f"  Embedding dimensions to test: {EMBEDDING_DIMENSIONS}")
print(f"  Test size: {TEST_SIZE * 100}%")
print(f"  Random state: {RANDOM_STATE}")
print(f"  Max training size: {MAX_TRAIN_SIZE}")

In [ ]:
# Use subset for faster experiments
if len(graphs) > MAX_TRAIN_SIZE / (1 - TEST_SIZE):
    print(f"\n🔄 Using subset of {int(MAX_TRAIN_SIZE / (1 - TEST_SIZE))} graphs for efficiency...")
    indices = np.random.RandomState(RANDOM_STATE).choice(
        len(graphs), int(MAX_TRAIN_SIZE / (1 - TEST_SIZE)), replace=False
    )
    graphs_subset = [graphs[i] for i in indices]
    labels_subset = labels[indices]
else:
    graphs_subset = graphs
    labels_subset = labels

# Split data
graphs_train, graphs_test, y_train, y_test = train_test_split(
    graphs_subset, labels_subset, test_size=TEST_SIZE, 
    random_state=RANDOM_STATE, stratify=labels_subset
)

print(f"✓ Train set: {len(graphs_train)} graphs")
print(f"✓ Test set: {len(graphs_test)} graphs")

## 8. Run Experiments with Different Embedding Dimensions

In [ ]:
# Store all results
all_results = []

for dim in EMBEDDING_DIMENSIONS:
    print(f"\n{'='*70}")
    print(f"🧪 EVALUATING WITH EMBEDDING DIMENSION: {dim}")
    print(f"{'='*70}")
    
    # Generate embeddings with memory tracking
    tracemalloc.start()
    start_time = time.time()
    
    print("⚡ Generating embeddings for training set...")
    model = FGSD(hist_bins=dim, hist_range=20, seed=RANDOM_STATE)
    model.fit(graphs_train)
    X_train = model.get_embedding()
    
    print("⚡ Generating embeddings for test set...")
    X_test = model.infer(graphs_test)
    
    generation_time = time.time() - start_time
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_mb = peak / 1024 / 1024
    
    print(f"\n📊 Embedding Statistics:")
    print(f"  Generation time: {generation_time:.2f}s")
    print(f"  Peak memory usage: {memory_mb:.2f} MB")
    print(f"  Train embeddings shape: {X_train.shape}")
    print(f"  Test embeddings shape: {X_test.shape}")
    
    # Check variance
    variance = np.var(X_train, axis=0)
    print(f"  Feature variance - Min: {np.min(variance):.4f}, Max: {np.max(variance):.4f}, Mean: {np.mean(variance):.4f}")
    
    # Define classifiers
    classifiers = {
        'SVM (Linear)': SVC(kernel='linear', C=1.0, random_state=RANDOM_STATE, probability=True),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE, n_jobs=-1),
        'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=15, 
                                               random_state=RANDOM_STATE, n_jobs=-1),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5,
                                                       random_state=RANDOM_STATE)
    }
    
    # Evaluate classifiers
    print(f"\n🎯 Evaluating Classifiers:")
    for clf_name, clf in classifiers.items():
        print(f"\n  Testing {clf_name}...", end=' ')
        result = evaluate_classifier(X_train, X_test, y_train, y_test, clf_name, clf)
        result['embedding_dim'] = dim
        result['generation_time'] = generation_time
        result['memory_mb'] = memory_mb
        all_results.append(result)
        
        print("✓")
        print(f"    Accuracy: {result['accuracy']:.4f}")
        print(f"    F1-Score: {result['f1_score']:.4f}")
        if result['auc'] is not None:
            print(f"    AUC: {result['auc']:.4f}")
        print(f"    Training time: {result['train_time']:.4f}s")
        print(f"    Inference time: {result['inference_time']:.4f}s")

print("\n✅ All experiments completed!")

## 9. Results Summary and Visualization

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame([{k: v for k, v in r.items() if k != 'y_pred'} for r in all_results])

print("\n" + "="*100)
print("📋 SUMMARY OF RESULTS")
print("="*100)
print(results_df.to_string(index=False))

# Save results
results_df.to_csv('/content/fgsd_reddit_results.csv', index=False)
print(f"\n💾 Results saved to /content/fgsd_reddit_results.csv")

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Accuracy comparison
ax = axes[0, 0]
for clf_name in results_df['classifier'].unique():
    clf_data = results_df[results_df['classifier'] == clf_name]
    ax.plot(clf_data['embedding_dim'], clf_data['accuracy'], marker='o', label=clf_name)
ax.set_xlabel('Embedding Dimension')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy vs Embedding Dimension')
ax.legend()
ax.grid(True, alpha=0.3)

# F1-Score comparison
ax = axes[0, 1]
for clf_name in results_df['classifier'].unique():
    clf_data = results_df[results_df['classifier'] == clf_name]
    ax.plot(clf_data['embedding_dim'], clf_data['f1_score'], marker='s', label=clf_name)
ax.set_xlabel('Embedding Dimension')
ax.set_ylabel('F1-Score')
ax.set_title('F1-Score vs Embedding Dimension')
ax.legend()
ax.grid(True, alpha=0.3)

# Training time comparison
ax = axes[1, 0]
for clf_name in results_df['classifier'].unique():
    clf_data = results_df[results_df['classifier'] == clf_name]
    ax.plot(clf_data['embedding_dim'], clf_data['train_time'], marker='^', label=clf_name)
ax.set_xlabel('Embedding Dimension')
ax.set_ylabel('Training Time (s)')
ax.set_title('Training Time vs Embedding Dimension')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# Memory usage
ax = axes[1, 1]
dims = results_df.groupby('embedding_dim')['memory_mb'].first()
ax.bar(dims.index, dims.values, color='steelblue', alpha=0.7)
ax.set_xlabel('Embedding Dimension')
ax.set_ylabel('Memory (MB)')
ax.set_title('Peak Memory Usage vs Embedding Dimension')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Best results
print("\n" + "="*100)
print("🏆 BEST RESULTS PER METRIC")
print("="*100)

best_acc = results_df.loc[results_df['accuracy'].idxmax()]
best_f1 = results_df.loc[results_df['f1_score'].idxmax()]
fastest_gen = results_df.loc[results_df['generation_time'].idxmin()]
fastest_train = results_df.loc[results_df['train_time'].idxmin()]

print(f"🥇 Best Accuracy: {best_acc['accuracy']:.4f}")
print(f"   Classifier: {best_acc['classifier']}")
print(f"   Embedding Dim: {best_acc['embedding_dim']}")

print(f"\n🥇 Best F1-Score: {best_f1['f1_score']:.4f}")
print(f"   Classifier: {best_f1['classifier']}")
print(f"   Embedding Dim: {best_f1['embedding_dim']}")

print(f"\n⚡ Fastest Embedding Generation: {fastest_gen['generation_time']:.2f}s")
print(f"   Embedding Dim: {fastest_gen['embedding_dim']}")

print(f"\n⚡ Fastest Training: {fastest_train['train_time']:.4f}s")
print(f"   Classifier: {fastest_train['classifier']}")
print(f"   Embedding Dim: {fastest_train['embedding_dim']}")

## 10. Confusion Matrix for Best Model

In [ ]:
# Get best model predictions
best_result = all_results[results_df['accuracy'].idxmax()]

# Plot confusion matrix
cm = confusion_matrix(y_test, best_result['y_pred'])
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title(f'Confusion Matrix - {best_result["classifier"]} (dim={best_result["embedding_dim"]})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Classification report
print("\n📊 Classification Report for Best Model:")
print(f"Classifier: {best_result['classifier']}")
print(f"Embedding Dimension: {best_result['embedding_dim']}")
print("\n" + classification_report(y_test, best_result['y_pred']))

## 11. Download Results

You can download the results CSV file from the Files panel on the left, or use the code below to download it directly.

In [ ]:
from google.colab import files

# Download results
files.download('/content/fgsd_reddit_results.csv')
print("✓ Results file ready for download!")